In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import torch
from sentence_transformers import SentenceTransformer

import faiss
import numpy as np
import pickle
from pathlib import Path
from vector_engine.utils import vector_search, id2details

#df = pd.read_csv('/content/Kittlerorakel/paragraphs.csv')
df = pd.read_csv('sentences.csv')

#df.head(3)
#print(f"Kittler paragraphs: {df.Id.unique().shape[0]}")

model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

embeddings = model.encode(df.Text.to_list(), show_progress_bar=True)
#print(f'Shape of the vectorised paragraphs: {embeddings[0].shape}')
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
index = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIDMap(index)
index.add_with_ids(embeddings, df.Id.values)

#print(f"Number of vectors in the Faiss index: {index.ntotal}")

# Paragraph
#df.iloc[5415, 2]

# Retrieve the 10 nearest neighbours
#D, I = index.search(np.array([embeddings[5415]]), k=10)
#print(f'L2 distance: {D.flatten().tolist()}\n\nParagraph Ids: {I.flatten().tolist()}')

# Fetch the original titles based on their index
#id2details(df, I, 'Title')

# Fetch the similar texts based on their index
#id2details(df, I, 'Text')

with open("models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)